# Cost Based Optimization

## Task: see how statistics are used

* turn CBO on
* save dataframe as table using metastore
* run simple query and see the query plan with stats using EXPLAIN COST
    * since Spark 3.0 we can use `explain(mode='cost')`
* run ANALYZE TABLE and see it again
* compute stats for individual cols and see the difference
* compute the histogram

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os

In [2]:
spark = (
    SparkSession
    .builder
    .appName('CBO')
    .config("spark.sql.hive.metastore.version", "1.2.1")
    .config("spark.sql.hive.metastore.jars", "maven")
    .enableHiveSupport()
    .getOrCreate()
)

#### Turn CBO on

In [3]:
spark.conf.set('spark.sql.cbo.enabled', True)

In [5]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

users_input_path = os.path.join(project_path, 'data/users')

users_output_path = os.path.join(project_path, 'output/users')

In [6]:
usersDF = spark.read.load(users_input_path)

#### See the query plan with stats

Hint:
* compose a query with filter user_id < -1000 (we know that there are no such records)
* use explain with mode='cost' to see the plan with stats

In [8]:
(
    usersDF
    .filter(col('user_id') < -1000)
).explain(mode='cost')

== Optimized Logical Plan ==
Filter (isnotnull(user_id#0L) AND (user_id#0L < -1000)), Statistics(sizeInBytes=4.1 MiB)
+- Relation[user_id#0L,display_name#1,about#2,location#3,downvotes#4L,upvotes#5L,reputation#6L,views#7L] parquet, Statistics(sizeInBytes=4.1 MiB)

== Physical Plan ==
*(1) Project [user_id#0L, display_name#1, about#2, location#3, downvotes#4L, upvotes#5L, reputation#6L, views#7L]
+- *(1) Filter (isnotnull(user_id#0L) AND (user_id#0L < -1000))
   +- *(1) ColumnarToRow
      +- FileScan parquet [user_id#0L,display_name#1,about#2,location#3,downvotes#4L,upvotes#5L,reputation#6L,views#7L] Batched: true, DataFilters: [isnotnull(user_id#0L), (user_id#0L < -1000)], Format: Parquet, Location: InMemoryFileIndex[file:/Users/david.vrba/spark-trainings/From-Simple-Transformations-to-Highly-Ef..., PartitionFilters: [], PushedFilters: [IsNotNull(user_id), LessThan(user_id,-1000)], ReadSchema: struct<user_id:bigint,display_name:string,about:string,location:string,downvotes:bigint,upvo

#### Now save the table using metastore

Hint
* use `saveAsTable()`

In [9]:
(
    usersDF
    .write
    .mode('overwrite')
    .option('path', users_output_path)
    .saveAsTable('users')
)

In [10]:
# see the tables we have:

spark.sql('show tables').show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|         data_summit|      false|
| default|data_summit_parti...|      false|
| default|      delta_table_v1|      false|
| default|      delta_table_v2|      false|
| default|     insert_table_v3|      false|
| default|           questions|      false|
| default|        questions_ss|      false|
| default|questions_ss_part...|      false|
| default|questions_ss_part...|      false|
| default|  spark_summit_users|      false|
| default|          test_table|      false|
| default|               users|      false|
+--------+--------------------+-----------+



#### See the query plan again

* Now read the table from metastore

In [12]:
(
    spark.table('users')
    .filter(col('user_id') < -1000)
).explain(mode='cost')

== Optimized Logical Plan ==
Filter (isnotnull(user_id#72L) AND (user_id#72L < -1000)), Statistics(sizeInBytes=4.1 MiB)
+- Relation[user_id#72L,display_name#73,about#74,location#75,downvotes#76L,upvotes#77L,reputation#78L,views#79L] parquet, Statistics(sizeInBytes=4.1 MiB)

== Physical Plan ==
*(1) Project [user_id#72L, display_name#73, about#74, location#75, downvotes#76L, upvotes#77L, reputation#78L, views#79L]
+- *(1) Filter (isnotnull(user_id#72L) AND (user_id#72L < -1000))
   +- *(1) ColumnarToRow
      +- FileScan parquet default.users[user_id#72L,display_name#73,about#74,location#75,downvotes#76L,upvotes#77L,reputation#78L,views#79L] Batched: true, DataFilters: [isnotnull(user_id#72L), (user_id#72L < -1000)], Format: Parquet, Location: InMemoryFileIndex[file:/Users/david.vrba/spark-trainings/From-Simple-Transformations-to-Highly-Ef..., PartitionFilters: [], PushedFilters: [IsNotNull(user_id), LessThan(user_id,-1000)], ReadSchema: struct<user_id:bigint,display_name:string,about:s

#### See the statistics for the table

Hint
* use sql DESC EXTENDED

In [13]:
spark.sql('DESC EXTENDED users').show(truncate=60)

+----------------------------+------------------------------------------------------------+-------+
|                    col_name|                                                   data_type|comment|
+----------------------------+------------------------------------------------------------+-------+
|                     user_id|                                                      bigint|   null|
|                display_name|                                                      string|   null|
|                       about|                                                      string|   null|
|                    location|                                                      string|   null|
|                   downvotes|                                                      bigint|   null|
|                     upvotes|                                                      bigint|   null|
|                  reputation|                                                      bigint|   null|


#### Compute the statistics

Hint
* run sql ANALYZE TABLE ... COMPUTE STATISTICS

In [14]:
spark.sql('ANALYZE TABLE users COMPUTE STATISTICS')

DataFrame[]

#### See the stats again

In [15]:
spark.sql('DESC EXTENDED users').show(truncate=60)

+----------------------------+------------------------------------------------------------+-------+
|                    col_name|                                                   data_type|comment|
+----------------------------+------------------------------------------------------------+-------+
|                     user_id|                                                      bigint|   null|
|                display_name|                                                      string|   null|
|                       about|                                                      string|   null|
|                    location|                                                      string|   null|
|                   downvotes|                                                      bigint|   null|
|                     upvotes|                                                      bigint|   null|
|                  reputation|                                                      bigint|   null|


#### See the query plan for the query again

In [ ]:
spark.sql('EXPLAIN COST select * from users where user_id < -1000').show(truncate=False)

#### See column level stats

Hint
* use DESC EXTENDED table_name, col_name

In [ ]:
spark.sql('DESC EXTENDED users user_id').show(truncate=60)

#### Compute column level stats

Hint:
* use ANALYZE TABLE table_name COMPUTE STATISTICS FOR COLUMNS col_names

In [ ]:
spark.sql('ANALYZE TABLE users COMPUTE STATISTICS FOR COLUMNS user_id, display_name')

#### See the stats again

In [ ]:
spark.sql('DESC EXTENDED users user_id').show(truncate=60)

#### See the plan again

In [ ]:
spark.sql('EXPLAIN COST select * from users where user_id < -1000').show(truncate=False)

#### Compute the histogram for specific cols

Hint
* Check if histogram is enabled
* Enable if not
* Compute column level stats again

In [ ]:
spark.conf.get('spark.sql.statistics.histogram.enabled')

In [ ]:
spark.conf.set('spark.sql.statistics.histogram.enabled', True)

In [ ]:
spark.sql('ANALYZE TABLE users COMPUTE STATISTICS FOR COLUMNS user_id')

#### See the stats again

In [ ]:
spark.sql('DESC EXTENDED users user_id').show(truncate=60)